The following is from [this article](https://medium.com/@pravash-techie/python-event-monitoring-with-watchdogs-86125f946da6) in Medium.

Since, It’s an open source automation file handling library, It will reduce the manual work as this file-watcher, upon the arrival/modification of any file, will trigger a follow-up process. Here, I will explain about one of the file event handler of this module with examples which will help you to get started with it. So, lets dive in

# 1. What is Watchdog?

watchdog is a Python API library and shell utilities to monitor file system events — based on the official documentation.

It provides a simple and intuitive interface for monitoring files and directories, and it can be used to automate various tasks that require you to keep an eye on changes to files and directories.

# 2. Event Handler and Observer

The main implementation or you can say the building blocks of watchdog is based on the following classes:

- Observer
- Event handler

The following imports are what all we require -

In [ ]:
from watchdog.events import FileSystemEventHandler
from watchdog.observers import Observer

## 2.1. Event handler

Below are the 4 Event Handlers available in `watchdog`:

- `FileSystemEventHandler`: Base file system event handler that you can override methods from.
- `PatternMatchingEventHandler`: Matches given patterns with file paths associated with occurring events.
- `RegexMatchingEventHandler`: Matches given regexes with file paths associated with occurring events.
- `LoggingEventHandler`: Logs all the events captured.

> You can refer to this [link](https://python-watchdog.readthedocs.io/en/stable/api.html#module-watchdog.events) for more details on Handlers

By extending an event handler class provided by `watchdog` we gain the ability to handle modified, created, deleted and moved events by implementing their respective [functions](https://pythonhosted.org/watchdog/api.html#event-classes) and also following functions which we can override as the other remaining classes inherits from `FileSystemEventHandler`

**on_any_event(event)**

**on_created(event)**

**on_deleted(event)**

**on_modified(event)**

**on_moved(event)**

For each of the functions, it will have an input parameter called event which contains the following variables:

`event_type` — The type of the event as a string. Default to None.

`is_directory` — True if event was emitted for a directory; False otherwise.

`src_path` — Source path of the file system object that triggered this event.

## 2.2. Observer

Since you all are familiar with the design patterns, So watchdog follows the observation design pattern.

So every observer will have the events and it will look and show the changes if there is any change in the files or directories.

Observer threads, schedules watching directories and dispatches calls to event handlers

> You can also import platform specific classes directly and use it instead of [`Observer`](https://python-watchdog.readthedocs.io/en/stable/api.html#watchdog.observers.Observer)

# 3. Understanding with implementation

So for this example I will be using the `FileSystemEventHandler` event class.
I will be setting the watch on a folder and trigger another function if any files arrive. And after the process is completed I will be moving that file to another folder.

- First of all, you need to create the instances of the observer and event handler which you be using to inherit the `FileSystemEventHandler`.

In [ ]:
from watchdog.events import FileSystemEventHandler
from watchdog.observers import Observer

In [ ]:
class MyHandler(FileSystemEventHandler):
    pass

In [ ]:
observer = Observer()
event_handler = MyHandler()

- Now, you need to schedule this observer. This observer will take the following 3 parameters -

   `event_handler`: The handler object you have created

   `path`: The folder you want to track

   `recursive`: How many times it should track

In [ ]:
observer.schedule(event_handler, path="./input_files", recursive=True)

- `observer.start()` — Starts the `Observer` thread and wait for it to generate events that will trigger the code inside the event handler. You can use the `time` to set the monitoring time (`time.sleep(100)`).

- `observer.stop()` this function will clean up the resources.

- Now finish this with the `observer.join()`, as we are using the multi threading concept here. Here the `join()` will join the multiple threads and until the thread whose join method is called terminates.

- Now moving to the event class. In this example, I will be checking if any file is uploaded into the tracking folder. for that I can use `on_created(event)`.

> Note: `watchdog` will also check the sub directories as well if you provide the parent directory

In [ ]:
import os
import shutil

In [ ]:
def create_directory(file_path=None):
    # Get the current date in the format of 'year-month-day'
    current_date = datetime.now().strftime("%Y-%m-%d")

    # Create a folder with the current date
    folder_path = f"{file_path}/{current_date}"
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        return folder_path
    else:
        return folder_path

In [ ]:
class MyHandler(FileSystemEventHandler):
    def on_created(self, event):
        dir_path = event.src_path.split("/input_files")
        processed_files = f"{dir_path[0]}/processed_files"

        child_processed_dir = create_directory(file_path=processed_files)

        if event:
            print(f"file created:{event.src_path}")
            # call function here
            main(file_name=event.src_path)

            file_name = event.src_path.split("/")[-1]
            destination_path = f"{child_processed_dir}/{file_name}"

            # Move file to other directory
            shutil.move(event.src_path, destination_path)
            print("file moved:{} to {}".format(event.src_path, destination_path))

In the example above, I am using function — `create_directory()`, to check if a folder with current date is in the destination path or else to create the same.
Then I am using the same path as the destination to move files (`shutil.move` module) after I did some processing in function — `main()`, from other python script.

Please find below the final code -

*event_handler_ex.py -*

In [ ]:
import os
import shutil
import time
from datetime import datetime

from watchdog.events import FileSystemEventHandler
from watchdog.observers import Observer
from watchdog_fileobserver_ex import main


def create_directory(file_path=None):
    # Get the current date in the format of 'year-month-day'
    current_date = datetime.now().strftime("%Y-%m-%d")

    # Create a folder with the current date
    folder_path = f"{file_path}/{current_date}"
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        return folder_path
    else:
        return folder_path


class MyHandler(FileSystemEventHandler):
    def on_created(self, event):
        dir_path = event.src_path.split("/input_files")
        processed_files = f"{dir_path[0]}/processed_files"

        child_processed_dir = create_directory(file_path=processed_files)

        if event:
            print("file created:{}".format(event.src_path))
            # call function here
            main(file_name=event.src_path)

            file_name = event.src_path.split("/")[-1]
            destination_path = f"{child_processed_dir}/{file_name}"

            shutil.move(event.src_path, destination_path)
            print("file moved:{} to {}".format(event.src_path, destination_path))


# if __name__ == "__main__":
#     observer = Observer()
#     event_handler = MyHandler()
#     observer.schedule(event_handler, path="./input_files", recursive=True)
#     observer.start()
#     try:
#         while True:
#             time.sleep(300)
#     except KeyboardInterrupt:
#         observer.stop()
#     observer.join()

*watchdog_fileobserver_ex.py -*

In [ ]:
import csv


def read_csv_file(file_name):
    try:
        with open(f"{file_name}", "r") as file:
            csvreader = csv.DictReader(file)
            for row in csvreader:
                print(row)
        return csvreader
    except Exception as e:
        pass


def main(file_name=None):
    if file_name:
        dict_data = read_csv_file(file_name)
        print("Process completed")
    else:
        print("Invalid file path")

## Wait for the File Transfer to Complete !!

Case where you need to wait for a file to be uploaded and then perform the required operation.
for that you can add either of the below code inside event functions -

# 4. Use cases of watchdog

## Monitoring a directory for changes:

The observer can be set up to watch the specified directory and all its subdirectories, and to call the appropriate method (`on_created`, `on_deleted`, or `on_modified`) whenever a file or directory is created, deleted, or modified.
The observer runs in an infinite loop, which can be interrupted by a keyboard interrupt.

In [12]:
import time
import os
from watchdog.events import FileSystemEventHandler
from watchdog.observers import Observer

In [13]:
class EventHandler(FileSystemEventHandler):
    def on_created(self, event):
        if event.is_directory:
            print("Directory created:", event.src_path)
        else:
            print("File created:", event.src_path)
            file_size = -1
            while file_size != os.path.getsize(event.src_path):
                file_size = os.path.getsize(event.src_path)
                print(file_size)
                time.sleep(1)

    def on_deleted(self, event):
        if event.is_directory:
            print("Directory deleted:", event.src_path)
        else:
            print("File deleted:", event.src_path)

    def on_modified(self, event):
        if event.is_directory:
            print("Directory modified:", event.src_path)
        else:
            print("File modified:", event.src_path)

In [14]:
event_handler = EventHandler()
observer = Observer()

In [15]:
observer.schedule(event_handler, "./input_files", recursive=True)

<ObservedWatch: path='./input_files', is_recursive=True>

In [16]:
observer.start()

File created: ./input_files\MCM-204_1_2023.9.6_17.7.27.csv
25261
File modified: ./input_files\MCM-204_1_2023.9.6_17.7.27.csv


In [ ]:
# try:
#     while True:
#         time.sleep(1)
# except KeyboardInterrupt:
#     observer.stop()
# observer.join()

In [17]:
observer.stop()